# Educational Kyber PKE (Toy Version)
This notebook demonstrates a simplified and insecure version of the Kyber public key encryption scheme for educational purposes only.

In [1]:
# Step 0: Imports
import numpy as np
from hashlib import sha3_256
from secrets import token_bytes

In [2]:
# Step 1: Parameter Definitions
q = 3329       # Prime modulus (same as Kyber)
n = 8          # Polynomial degree (use 256 in real Kyber)
eta = 2        # Noise distribution bound (uniform [-eta, eta])
seed_bytes = 32

In [3]:
# Step 2: Polynomial Arithmetic
def poly_add(a, b):
    return (a + b) % q

def poly_sub(a, b):
    return (a - b) % q

def poly_mul(a, b):
    res = np.zeros(2 * n - 1, dtype=int)
    for i in range(n):
        for j in range(n):
            res[i + j] += a[i] * b[j]
    for i in range(n, 2 * n - 1):
        res[i - n] = (res[i - n] - res[i]) % q
    return res[:n] % q

In [4]:
# Step 3: Noise Sampling
def sample_noise():
    return np.random.randint(-eta, eta + 1, size=n) % q

In [5]:
# Step 4: Key Generation
def keygen():
    A = np.random.randint(0, q, size=(n, n))
    s = sample_noise()
    e = sample_noise()
    b = (A @ s + e) % q
    return (A, b), s

In [6]:
# Step 5: Encryption
def encrypt(pk, m):
    A, b = pk
    m_poly = np.array([q // 2 if bit == '1' else 0 for bit in m] + [0] * (n - len(m)))
    r = sample_noise()
    e1 = sample_noise()
    e2 = sample_noise()
    u = (A.T @ r + e1) % q
    v = (np.dot(b, r) + e2 + m_poly) % q
    return u, v

In [7]:
# Step 6: Decryption
def decrypt(sk, ciphertext):
    u, v = ciphertext
    s = sk
    m_rec = (v - np.dot(u, s)) % q
    return ''.join(['1' if val > q//4 and val < 3*q//4 else '0' for val in m_rec[:4]])

In [12]:
# Step 7: Demonstration
pk, sk = keygen()
message = '1011'
ciphertext = encrypt(pk, message)
decrypted = decrypt(sk, ciphertext)

print("Original Message: ", message)
print("Decrypted Message:", decrypted)

Original Message:  1011
Decrypted Message: 1011
